We will follow 4 steps to build this agentic system-
1. First we ensure the neccessary frameworks and libraries are installed.
2. Next, we will write functions to load the resumes.
3. Then we use CrewAI to define agents and their respective tasks.
4. And finally we construct the crew to bundle all the agents and their tasks in a flow and review the output.

# Step 0: Install and Import Relevant Libraries

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
#!pip install PyMuPDF
#!pip install python-docx
#!pip install crewai crewai-tools

# Step 1: Loading the Resume

In [3]:
import fitz  # PyMuPDF for PDF processing
import docx  # python-docx for DOCX processing

In [5]:
def extract_text_from_pdf(file_path):
    """Extracts text from a PDF file using PyMuPDF."""
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def extract_text_from_docx(file_path):
    """Extracts text from a DOCX file using python-docx."""
    doc = docx.Document(file_path)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return "\n".join(fullText)

def extract_text_from_resume(file_path):
    """Determines file type and extracts text."""
    if file_path.endswith(".pdf"):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith(".docx"):
        return extract_text_from_docx(file_path)
    else:
        return "Unsupported file format."

In [11]:
res1 = extract_text_from_resume('Anita_Sanjok_Resume.pdf')
print(res1)

Anita Sanjok
Dwarka, 110078 | 123-456-7890 | anitaname@delhiuni.edu | LinkedIn
Professional Title: Youth Development Specialist
Summary
Dedicated Youth Development Specialist with a passion for fostering self-efficacy and confidence in
adolescents.
Proven ability to design and implement educational programs that connect youth with critical
resources and
opportunities to enhance personal growth and academic success. Excellent collaborator,
communicator, and data
analyst, focused on driving program effectiveness through insightful evaluation.
Education
University of Delhi, Delhi - Master of Arts in Education (Expected May 2025)
Relevant Coursework: Child Psychology, Educational Leadership, Curriculum Development
Amity University, Jaipur - Bachelor of Arts in Sociology (May 2018)
Projects: Conducted qualitative research on community engagement in education
Relevant Professional Experience
University of Delhi, Vishwavidyalaya, Delhi - Youth Development Specialist / Research Assistant
(Jan 

In [12]:
res2 = extract_text_from_resume('Resume_Agent_V2_R.ipynb')
print(res2)

Unsupported file format.


# Step 2: Preparing the Agents and Task

- Agent represents an AI or human assistant with a specific role and goal.
- Task defines an objective that an agent needs to accomplish.
- And finally, Crew is used to bundle multiple agents and their respective tasks together to work and achieve the set objective.


In [14]:
!pip install crewai


  Using cached click-8.3.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typer-0.20.0-py3-none-any.whl.metadata (16 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached rich-14.2.0-py3-none-any.whl.metadata (18 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached charset_normalizer-3.4.4-cp311-cp311-win_amd64.whl.metadata (38 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached cert

In [15]:
import os
from crewai import Agent, Task, Crew

In [16]:
with open('openai.txt', 'r') as file:
    openai_key = file.read()
    
os.environ['OPENAI_API_KEY'] = openai_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

- The first agent will provide feedback on the resume.
- The second agent will improve the resume based on the feedback
- And, the third and final agent would suggest appropriate jobs based on the the improved resume and specified location

In [17]:
# Agent 1: Resume Strategist
resume_feedback = Agent(
    role="Professional Resume Advisor",
    goal="Give feedback on the resume to make it stand out in the job market.",
    verbose=True,
    backstory="With a strategic mind and an eye for detail, you excel at providing feedback on resumes to highlight the most relevant skills and experiences."
    )

In [18]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_feedback_task = Task(
    description=(
        """Give feedback on the resume to make it stand out for recruiters. 
        Review every section, inlcuding the summary, work experience, skills, and education. Suggest to add relevant sections if they are missing.  
        Also give an overall score to the resume out of 10.  This is the resume: {resume}"""
    ),
    expected_output="The overall score of the resume followed by the feedback in bullet points.",
    agent=resume_feedback
)

In [20]:
# Agent 2: Resume Strategist
resume_advisor = Agent(
    role="Professional Resume Writer",
    goal="Based on the feedback recieved from Resume Advisor, make changes to the resume to make it stand out in the job market.",
    verbose=True,
    backstory="With a strategic mind and an eye for detail, you excel at refining resumes based on the feedback to highlight the most relevant skills and experiences."
)

In [21]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_advisor_task = Task(
    description=(
        """Rewrite the resume based on the feedback to make it stand out for recruiters. You can adjust and enhance the resume but don't make up facts. 
        Review and update every section, including the summary, work experience, skills, and education to better reflect the candidates abilities. This is the resume: {resume}"""
    ),
    expected_output= "Resume in markdown format that effectively highlights the candidate's qualifications and experiences",
    # output_file="improved_resume.md",
    context=[resume_feedback_task],
    agent=resume_advisor
)

In [23]:
!pip install crewai-tools


   ---------------------------------------- 0.0/755.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/755.0 kB ? eta -:--:--
   ------------- -------------------------- 262.1/755.0 kB ? eta -:--:--
   -------------------------- ----------- 524.3/755.0 kB 840.2 kB/s eta 0:00:01
   ---------------------------------------- 755.0/755.0 kB 957.0 kB/s  0:00:00
   ---------------------------------------- 0.0/41.1 MB ? eta -:--:--
    --------------------------------------- 0.8/41.1 MB 3.7 MB/s eta 0:00:11
   - -------------------------------------- 1.6/41.1 MB 4.6 MB/s eta 0:00:09
   -- ------------------------------------- 2.4/41.1 MB 3.9 MB/s eta 0:00:10
   --- ------------------------------------ 3.4/41.1 MB 4.4 MB/s eta 0:00:09
   ---- ----------------------------------- 4.5/41.1 MB 4.3 MB/s eta 0:00:09
   ----- ---------------------------------- 6.0/41.1 MB 4.9 MB/s eta 0:00:08
   ------- -------------------------------- 7.3/41.1 MB 5.1 MB/s eta 0:00:07
   -------- -----

In [24]:
from crewai_tools import SerperDevTool

In [25]:
with open('serper.txt', 'r') as file:
    serper_key = file.read()

os.environ["SERPER_API_KEY"] = serper_key

search_tool = SerperDevTool()

In [26]:
# Agent 3: Researcher
job_researcher = Agent(
    role = "Senior Recruitment Consultant",
    goal = "Find the 5 most relevant, recently posted jobs based on the improved resume recieved from resume advisor and the location preference",
    tools = [search_tool],
    verbose = True,
    backstory = """As a senior recruitment consultant your prowess in finding the most relevant jobs based on the resume and location preference is unmatched. 
    You can scan the resume efficiently, identify the most suitable job roles and search for the best suited recently posted open job positions at the preffered location."""
    )

In [27]:
research_task = Task(
    description = """Find the 5 most relevant recent job postings based on the resume recieved from resume advisor and location preference. This is the preferred location: {location} . 
    Use the tools to gather relevant content and shortlist the 5 most relevant, recent, job openings""",
    expected_output=(
        "A bullet point list of the 5 job openings, with the appropriate links and detailed description about each job, in markdown format" 
    ),
#    output_file="relevant_jobs.md",
    agent=job_researcher
)

# Step 3: Creating the Crew and Reviwing the Output

In [28]:
crew = Crew(
    agents=[resume_feedback, resume_advisor, job_researcher],
    tasks=[resume_feedback_task, resume_advisor_task, research_task],
    verbose=True
)

In [29]:
result = crew.kickoff(inputs={"resume": res2, "location": 'New Delhi'})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 3c7a00a6-7866-4768-adfd-f50172ead194                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Professional Resume Advisor                                                                             │
│                                                                                                                 │
│  Task: Give feedback on the resume to make it stand out for recruiters.                                         │
│          Review every section, inlcuding the summary, work experience, skills, and education. Suggest to add    │
│  relevant sections if they are missing.                                                                         │
│          Also give an overall score to the resume out of 10.  This is the resume: Unsupported file format.      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Professional Resume Advisor                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Overall Score: 4/10                                                                                            │
│                                                                                                                 │
│  Feedback:                                                                                                      │
│  - **Summary Section:**                                                                                         │
│    - Consider adding a professional summary at the top that highlights your key achievements and career goals.  │
│  Tailor it to the specific job you are applying for to make a strong first impression.                          │
│                                                                                                                 │
│  - **Work Experience Section:**                                                                                 │
│    - Ensure each position includes measurable achievements and specific contributions (e.g., "increased sales   │
│  by 20%" or "improved process efficiency by reducing turnaround time by 15%"). Use action verbs to make your    │
│  contributions more impactful.                                                                                  │
│    - Include relevant internships or volunteer experiences that may showcase your skills and dedication,        │
│  especially if you have limited work experience in the field.                                                   │
│                                                                                                                 │
│  - **Skills Section:**                                                                                          │
│    - Include both hard and soft skills that are relevant to the job you want. Be specific (e.g., "Proficient    │
│  in Microsoft Excel, strong communication skills, and project management expertise").                           │
│    - Consider creating separate sections for technical skills and interpersonal skills for better visibility.   │
│                                                                                                                 │
│  - **Education Section:**                                                                                       │
│    - If you have advanced degrees or certifications, make sure they are prominently displayed, and include any  │
│  honors or distinctions received.                                                                               │
│    - Add coursework or projects that relate directly to the job you are applying for if you are a recent        │
│  graduate.                                                                                                      │
│                                                                                                                 │
│  - **Formatting:**                                                                                              │
│    - Ensure the resume is easy to read with consistent font styles and sizes. Use bullet points for clarity     │
│  and to allow recruiters to scan through faster.                                                                │
│    - Keep the layout clean and professional. Avoid cluttering with excessive graphics or colors that could      │
│  detract from the content.                             

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 0552001c-3d33-4f32-92ef-303f4190ad14                                                                     │
│  Agent: Professional Resume Advisor                                                                             │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Professional Resume Writer                                                                              │
│                                                                                                                 │
│  Task: Rewrite the resume based on the feedback to make it stand out for recruiters. You can adjust and         │
│  enhance the resume but don't make up facts.                                                                    │
│          Review and update every section, including the summary, work experience, skills, and education to      │
│  better reflect the candidates abilities. This is the resume: Unsupported file format.                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Recruitment Consultant                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  - **[Customer Project Manager](https://www.naukri.com/customer-project-manager-jobs-in-delhi-ncr)**            │
│    Customer Project Manager jobs in Delhi NCR; Experience 3-7 years. The role involves managing customer        │
│  relations and project delivery ensuring client satisfaction.                                                   │
│                                                                                                                 │
│  - **[Project Management Business                                                                               │
│  Analyst](https://in.indeed.com/q-project-management-business-analyst-l-delhi-jobs.html)**                      │
│    274 Project Management Business Analyst jobs available in Delhi on Indeed.com. This position requires        │
│  strong analytical skills and experience in project management practices.                                       │
│                                                                                                                 │
│  - **[Business Analyst Project                                                                                  │
│  Management](https://in.linkedin.com/jobs/business-analyst-project-management-jobs-delhi)**                     │
│    31 Jobs available; this role focuses on improving processes and assisting in project management with a       │
│  strong analytical background. Recently added positions with competitive salaries.                              │
│                                                                                                                 │
│  - **[Project Management                                                                                        │
│  Consultant](https://www.glassdoor.co.in/Job/new-delhi-project-management-consultant-jobs-SRCH_IL.0,9_IC289168  │
│  1_KO10,39.htm)**                                                                                               │
│    1,155 open positions for Project Management Consultants in New Delhi. The role involves guiding projects     │
│  effectively ensuring that they meet both client and organization goals.                                        │
│                                                                                                                 │
│  - **[Business Analysis and Project Management                                                                  │
│  Jobs](https://www.hirist.tech/business-analysis-and-project-management-jobs-in-delhi-ncr)**                    │
│    Apply to 232 Business Analysis and Project Management Jobs in Delhi NCR; these positions require skills in   │
│  both business analysis and managing projects efficiently.                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 9f28bbba-bb2e-4f9a-998a-286f9b7cb31b                                                                     │
│  Agent: Senior Recruitment Consultant                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



┌───────────────────────────── Execution Traces ──────────────────────────────┐
│                                                                             │
│  🔍 Detailed execution traces are available!                                │
│                                                                             │
│  View insights including:                                                   │
│    • Agent decision-making process                                          │
│    • Task execution flow and timing                                         │
│    • Tool usage details                                                     │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
Would you like to view your execution traces? [y/N] (20s timeout): 

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 3c7a00a6-7866-4768-adfd-f50172ead194                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: - **[Customer Project                                                                            │
│  Manager](https://www.naukri.com/customer-project-manager-jobs-in-delhi-ncr)**                                  │
│    Customer Project Manager jobs in Delhi NCR; Experience 3-7 years. The role involves managing customer        │
│  relations and project delivery ensuring client satisfaction.                                                   │
│                                                                                                                 │
│  - **[Project Management Business                                                                               │
│  Analyst](https://in.indeed.com/q-project-management-business-analyst-l-delhi-jobs.html)**                      │
│    274 Project Management Business Analyst jobs available in Delhi on Indeed.com. This position requires        │
│  strong analytical skills and experience in project management practices.                                       │
│                                                                                                                 │
│  - **[Business Analyst Project                                                                                  │
│  Management](https://in.linkedin.com/jobs/business-analyst-project-management-jobs-delhi)**                     │
│    31 Jobs available; this role focuses on improving processes and assisting in project management with a       │
│  strong analytical background. Recently added positions with competitive salaries.                              │
│                                                                                                                 │
│  - **[Project Management                                                                                        │
│  Consultant](https://www.glassdoor.co.in/Job/new-delhi-project-management-consultant-jobs-SRCH_IL.0,9_IC289168  │
│  1_KO10,39.htm)**                                                                                               │
│    1,155 open positions for Project Management Consultants in New Delhi. The role involves guiding projects     │
│  effectively ensuring that they meet both client and organization goals.                                        │
│                                                                                                                 │
│  - **[Business Analysis and Project Management                                                                  │
│  Jobs](https://www.hirist.tech/business-analysis-and-project-management-jobs-in-delhi-ncr)**                    │
│    Apply to 232 Business Analysis and Project Management Jobs in Delhi NCR; these positions require skills in   │
│  both business analysis and managing projects efficiently.                                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



┌───────────────────────────── Execution Traces ──────────────────────────────┐
│                                                                             │
│  🔍 Detailed execution traces are available!                                │
│                                                                             │
│  View insights including:                                                   │
│    • Agent decision-making process                                          │
│    • Task execution flow and timing                                         │
│    • Tool usage details                                                     │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
Would you like to view your execution traces? [y/N] (20s timeout): 

In [ ]:
from IPython.display import Markdown, display

In [38]:
markdown_content = resume_feedback_task.output.raw.strip("```markdown").strip("```").strip()
# Display the Markdown content
display(Markdown(markdown_content))

Overall Score: 7/10  

Feedback:  
- **Contact Information**: Ensure your LinkedIn URL is hyperlinked in digital copies for easy access. Consider adding a professional title or a personal brand statement under your name to clarify your specialization (e.g., "Youth Development Specialist").  

- **Summary Section**: There is currently no summary section. Adding a summary can provide a snapshot of your expertise, experiences, and career goals. Aim for 2-3 impactful sentences that communicate your key strengths and values related to education and youth development.  

- **Education**: Your education section is clear and well-structured. You might want to include any relevant coursework or projects that support your expertise in education and youth development.  

- **Professional Experience**: 
  - Your bullet points are generally strong, but consider starting each with powerful action verbs (e.g., "Facilitated," "Collaborated," "Managed") to make your contributions stand out more.  
  - In your most recent role, quantify your achievements where possible (e.g., "Facilitated activities that improved student confidence by X% as measured by...").  
  - For your role as Program Coordinator, clarify your specific contributions and the impact of your workshops (e.g., "Resulted in a X% increase in college applications from participants").  

- **Additional Work Experience**: This section demonstrates your relevant experience, but more focus on accomplishments rather than responsibilities can enhance impact. For example, specify the number of workshops held or the percentage increase in student participation resulting from your initiatives.  

- **Skills Section**: This section looks good, but consider categorizing skills into both hard and soft skills (e.g., Project Management, Data Analysis, Communication). Also, highlight any specific interpersonal skills that reflect your ability to work with youth, such as mentorship or motivational skills.  

- **Volunteer Experience**: If applicable, consider adding a section for volunteer work. Experience in volunteer roles can demonstrate initiative and commitment to community and youth development.  

- **Formatting and Design**: Ensure consistency in fonts, bullet points, and spacing throughout the document. A more visually appealing format can enhance readability. Consider using a professional template to create a more polished look.  

By implementing the above suggestions, you can enhance the effectiveness of your resume, making it more appealing to recruiters and showcasing your qualifications comprehensively.

In [32]:
from IPython.display import Markdown, display

markdown_content = research_task.output.raw.strip("```markdown").strip("```").strip()
display(Markdown(markdown_content))


- **[Customer Project Manager](https://www.naukri.com/customer-project-manager-jobs-in-delhi-ncr)**  
  Customer Project Manager jobs in Delhi NCR; Experience 3-7 years. The role involves managing customer relations and project delivery ensuring client satisfaction.
  
- **[Project Management Business Analyst](https://in.indeed.com/q-project-management-business-analyst-l-delhi-jobs.html)**  
  274 Project Management Business Analyst jobs available in Delhi on Indeed.com. This position requires strong analytical skills and experience in project management practices.
  
- **[Business Analyst Project Management](https://in.linkedin.com/jobs/business-analyst-project-management-jobs-delhi)**  
  31 Jobs available; this role focuses on improving processes and assisting in project management with a strong analytical background. Recently added positions with competitive salaries.
  
- **[Project Management Consultant](https://www.glassdoor.co.in/Job/new-delhi-project-management-consultant-jobs-SRCH_IL.0,9_IC2891681_KO10,39.htm)**  
  1,155 open positions for Project Management Consultants in New Delhi. The role involves guiding projects effectively ensuring that they meet both client and organization goals.
  
- **[Business Analysis and Project Management Jobs](https://www.hirist.tech/business-analysis-and-project-management-jobs-in-delhi-ncr)**  
  Apply to 232 Business Analysis and Project Management Jobs in Delhi NCR; these positions require skills in both business analysis and managing projects efficiently.

In [33]:
markdown_content = resume_advisor_task.output.raw.strip("```markdown").strip("```").strip()
# Display the Markdown content
display(Markdown(markdown_content))

# [Your Name]  
[Your Address]  
[City, State, Zip]  
[Your Email] | [Your Phone Number] | [LinkedIn Profile]

---

## Professional Summary
Results-driven professional with over [X years] of experience in [Your Industry/Field]. Proven track record of [describe key achievements or skills relevant to the job, e.g., "increasing sales by 20% through strategic marketing initiatives"]. Highly skilled in [mention relevant skills, e.g., project management, customer relations, etc.], with a passion for driving results and enhancing team performance. Eager to leverage my expertise in [specific area relevant to job] to contribute to [Company Name]’s success.

---

## Work Experience  

**[Your Job Title]**  
**[Company Name, City, State]**  
**[Month, Year] – Present**  
- Led initiatives that increased [specific metric, e.g. sales, customer engagement] by [percentage or amount], resulting in [describe outcome].  
- Streamlined processes, achieving [specific improvement, e.g. 15% reduction in turnaround time], enhancing team efficiency.  
- Collaborated with cross-functional teams to implement solutions that improved overall service delivery.  

**[Your Previous Job Title]**  
**[Company Name, City, State]**  
**[Month, Year] – [Month, Year]**  
- Developed and executed [describe project or initiative], which resulted in [measurable outcome, e.g. cost savings, revenue growth].  
- Trained and mentored junior staff, improving team performance and enhancing skills across departments.  
- Analyzed performance metrics and utilized data to inform strategic decisions, leading to [specific advancement or outcome].  

**Internship/Volunteer Experience**  
**[Title/Role]**  
**[Organization Name, City, State]**  
**[Month, Year] – [Month, Year]**  
- Assisted with [describe key responsibilities], supporting the team to achieve [outcome].  
- Conducted research that contributed to [describe project or initiative], showcasing analytical abilities and dedication to the mission.  

---

## Skills  

### Technical Skills  
- Proficient in Microsoft Office Suite (Excel, Word, PowerPoint)  
- Familiar with [specific software/tools relevant to the job]  
- [Programming languages, technical writing, data analysis tools, etc.]  

### Interpersonal Skills  
- Strong communication and presentation abilities  
- Excellent problem-solving and critical-thinking skills  
- Proven ability to work collaboratively in diverse teams  

---

## Education  

**[Degree/Program]**  
**[University Name, City, State]**  
**[Month, Year] of Graduation**  
- Relevant Coursework: [list relevant courses]  
- Honors: [mention any honors or distinctions]  

---

## Certifications  
- [Certification Name, Year]  
- [Certification Name, Year]  

---

## Additional Experience  
**Workshops/Seminars**: Participated in [name or topic of relevant workshops or seminars]  
**Professional Organizations**: Member of [name of organizations relevant to your field]  
**Volunteer Work**: [Briefly describe volunteer work that aligns with your career goals]  

---

*References available upon request.*  
```

This rewritten resume presents a professional image while highlighting measurable achievements and relevant skills tailored to attract recruiters. Use this format as a foundation for customizing the content as needed for specific job applications.

In [34]:
markdown_content = research_task.output.raw.strip("```markdown").strip("```").strip()
# Display the Markdown content
display(Markdown(markdown_content))


- **[Customer Project Manager](https://www.naukri.com/customer-project-manager-jobs-in-delhi-ncr)**  
  Customer Project Manager jobs in Delhi NCR; Experience 3-7 years. The role involves managing customer relations and project delivery ensuring client satisfaction.
  
- **[Project Management Business Analyst](https://in.indeed.com/q-project-management-business-analyst-l-delhi-jobs.html)**  
  274 Project Management Business Analyst jobs available in Delhi on Indeed.com. This position requires strong analytical skills and experience in project management practices.
  
- **[Business Analyst Project Management](https://in.linkedin.com/jobs/business-analyst-project-management-jobs-delhi)**  
  31 Jobs available; this role focuses on improving processes and assisting in project management with a strong analytical background. Recently added positions with competitive salaries.
  
- **[Project Management Consultant](https://www.glassdoor.co.in/Job/new-delhi-project-management-consultant-jobs-SRCH_IL.0,9_IC2891681_KO10,39.htm)**  
  1,155 open positions for Project Management Consultants in New Delhi. The role involves guiding projects effectively ensuring that they meet both client and organization goals.
  
- **[Business Analysis and Project Management Jobs](https://www.hirist.tech/business-analysis-and-project-management-jobs-in-delhi-ncr)**  
  Apply to 232 Business Analysis and Project Management Jobs in Delhi NCR; these positions require skills in both business analysis and managing projects efficiently.

# Conclusion

- Build a fourth agent that customizes the resume, based on the job you select.
- Build another agent in the end to generate a cover letter customized for the job you choose to apply,
- Build an agent that will help you prepare for the job you select.

In [35]:
#additional project for mine resume based job selection
from crewai import Agent, Task

# Ask user for job preference
job_preference = input("Enter your preferred job role (e.g., Data Analyst, Software Engineer, etc.): ").strip()


In [36]:
# 1️⃣ Define the new agent
cover_letter_agent = Agent(
    role="Cover Letter Writer",
    goal=f"Write a professional, customized cover letter for the job role: {job_preference}",
    backstory=(
        "An expert career consultant and HR professional who crafts personalized cover letters "
        "that align a candidate’s achievements with job requirements."
    )
)

In [37]:
# 2️⃣ Define the task
cover_letter_task = Task(
    description=(
        f"Using the extracted resume details and the job preference '{job_preference}', "
        "generate a professional cover letter highlighting the candidate’s strengths, experience, and fit for the role. "
        "Ensure it sounds human, confident, and specific to the chosen job title."
    ),
    agent=cover_letter_agent,
    expected_output="A complete, well-structured cover letter ready to send to the employer."
)

In [43]:
# 3️⃣ Run the task using Crew
print("\n🚀 Generating personalized cover letter...\n")

cover_letter_crew = Crew(
    agents=[cover_letter_agent],
    tasks=[cover_letter_task]
)

cover_letter_result = cover_letter_crew.kickoff()


🚀 Generating personalized cover letter...



In [44]:
display(Markdown(f"### ✉️ Generated Cover Letter for *{job_preference}*\n\n{cover_letter_result}"))

### ✉️ Generated Cover Letter for *data nalyst*

[Your Name]  
[Your Address]  
[City, State, Zip]  
[Your Email]  
[Your Phone Number]  
[Date]  

[Hiring Manager's Name]  
[Company Name]  
[Company Address]  
[City, State, Zip]  

Dear [Hiring Manager's Name],  

I am writing to express my interest in the Data Analyst position at [Company Name], as advertised on [where you found the job listing]. With a solid foundation in data analysis, strong problem-solving abilities, and a passion for uncovering insights through data, I am excited about the opportunity to contribute to your team and support the key initiatives of [Company Name].

In my previous role at [Your Previous Company], I successfully analyzed large datasets, identifying trends and actionable insights that contributed to an increase in operational efficiency by 25%. Utilizing advanced analytical tools such as Python and SQL, I developed models that forecasted customer behavior, enhancing our marketing strategies and ultimately increasing customer retention rates by 15%. My proficiency in data visualization tools such as Tableau enabled me to present complex data in a clear and convincing manner, which facilitated strategic decision-making across various departments.

I hold a [Your Degree] in [Your Major] from [Your University], where I honed my analytical skills and gained hands-on experience through projects that involved data manipulation and analysis using various statistical methodologies. My academic background, combined with my [X years] of practical experience in data analytics, equips me with a unique perspective on leveraging data to drive transformative business solutions.

Additionally, I am particularly drawn to [Company Name]'s commitment to innovation and data-driven decision-making. I am eager to collaborate with cross-functional teams, utilizing my communication skills to convey complex findings and contribute to the organization’s goals effectively. My detail-oriented approach and ability to adapt to new technologies quickly will allow me to excel in the fast-paced environment at [Company Name].

I am thrilled at the prospect of being part of your team and contributing to the vital work [Company Name] is doing. Thank you for considering my application. I look forward to the opportunity to discuss how my skills and experiences align with the needs of your team in further detail.

Warmest regards,  

[Your Name]

In [47]:
with open("Generated_Cover_Letter.txt", "w", encoding="utf-8") as f:
   f.write(cover_letter_result.raw if hasattr(cover_letter_result, "raw") else str(cover_letter_result))

print("\n💾 Cover letter saved as 'Generated_Cover_Letter.txt'")


💾 Cover letter saved as 'Generated_Cover_Letter.txt'
